In [55]:
import numpy as np
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import torch
from calflops import calculate_flops
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T
from sophius.encode import Encoder

VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

encoder = Encoder()

Files already downloaded and verified


In [57]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(encoder.encode_model(model_tmpl))

Conv2d       (32, 16, 16)   (1, 1)   (2, 2)  
PReLU        (32, 16, 16)  
MaxPool2d    (32, 7, 7)     (4, 4)   (2, 2)  
Flatten      1568          
Linear       10            

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [58]:
model_tmpl = model_gen.generate_model_tmpl()
vec = encoder.encode_model(model_tmpl)
vec.dtype

dtype('uint8')

In [59]:
import numpy as np

model_tmpl = model_gen.generate_model_tmpl()
arr = encoder.encode_model(model_tmpl)

hex_str = encoder.encode_model_str(model_tmpl)

print(arr)
print(hex_str)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
008100890028d000400000001004000001000000


In [60]:
import sqlite3
import pandas as pd
import seaborn as sns
from tqdm import tqdm

loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

df = pd.DataFrame(columns=['hex_str', 'flops', 'macs', 'params', 'time', 'val_acc', 'train_acc'])

for i in tqdm(range(10)):
    model_tmpl = model_gen.generate_model_tmpl()
    model = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
    flops, macs, params = calculate_flops(model=model, 
                                      input_shape=(1, 3, 32, 32),
                                      output_as_string=False,
                                      print_results=False,
                                      print_detailed=False,
                                      output_precision=4)
    # express train
    t, val_acc, train_acc = train_express_gpu(model = model,
                                              train = True,
                                              loader = loader_gpu,
                                              milestones = [],
                                              num_epoch = 1,
                                              verbose = False)
    
    hex_str = encoder.encode_model_str(model_tmpl)
    
    df.loc[i] = [hex_str, flops, macs, params, t, val_acc, train_acc]
    
    with sqlite3.connect('../data/models.db') as conn:
        df.loc[i:i].to_sql('test', conn, if_exists='append', index=False)
    
    # print(df.loc[i])

100%|██████████| 10/10 [00:40<00:00,  4.00s/it]


In [65]:
df

,hex_str,flops,macs,params,time,val_acc,train_acc
0,0088012120000000004260000028c80000805006004320...,996128,486016,13674,17.115060,0.312500,0.308594
1,0081040540000000080400000049200000253000008802...,312672,143440,10211,11.852894,0.257812,0.262695
2,0080408a40000000080400000084100a40000000100200...,3474496,1705632,16475,7.240571,0.267578,0.260742
3,0081100940000000004510002000000001000000,186304,75072,52234,0.581247,0.399414,0.417969
4,00804206100200000010000001000000,1028992,498944,7562,0.542690,0.182617,0.171875
5,0088082140000000100200000010000001000000,223112,92336,210,0.537298,0.155273,0.133789
6,00840205002530001002000001000000,112448,54912,6554,0.112405,0.291992,0.277344
7,008802054000000020000000004910000010000001000000,52264,24656,498,0.146999,0.106445,0.129883
8,008028050028b00040000000100800000081100a400000...,433408,199168,17930,0.545936,0.302734,0.308594
9,0080208a00429000100200000082102200431000001000...,5265952,2607680,27434,0.957097,0.295898,0.268555


In [66]:
with sqlite3.connect('../data/models.db') as conn:
    df = pd.read_sql('SELECT * FROM test', conn)
df

,hex_str,flops,macs,params,time,val_acc,train_acc
0,0088012120000000004260000028c80000805006004320...,996128,486016,13674,17.115060,0.312500,0.308594
1,0081040540000000080400000049200000253000008802...,312672,143440,10211,11.852894,0.257812,0.262695
2,0080408a40000000080400000084100a40000000100200...,3474496,1705632,16475,7.240571,0.267578,0.260742
3,0081100940000000004510002000000001000000,186304,75072,52234,0.581247,0.399414,0.417969
4,00804206100200000010000001000000,1028992,498944,7562,0.542690,0.182617,0.171875
5,0088082140000000100200000010000001000000,223112,92336,210,0.537298,0.155273,0.133789
6,00840205002530001002000001000000,112448,54912,6554,0.112405,0.291992,0.277344
7,008802054000000020000000004910000010000001000000,52264,24656,498,0.146999,0.106445,0.129883
8,008028050028b00040000000100800000081100a400000...,433408,199168,17930,0.545936,0.302734,0.308594
9,0080208a00429000100200000082102200431000001000...,5265952,2607680,27434,0.957097,0.295898,0.268555
